# Movie Studio Analysis: Understanding Box Office Success
## Business Understanding
The company is planning to launch a new movie studio, but lacks experience in movie production. The goal of this analysis is to explore current trends in the film industry and provide actionable insights that can guide the studio's strategy. This analysis focuses on identifying factors that contribute to box office success, including genre, budget, release timing, and the impact of key personnel such as directors, actors, and writers.
    

## Data Understanding
The analysis is based on multiple datasets related to movie budgets, revenues, genres, release dates, and key personnel (directors, actors, and writers).

These datasets include:
   - **Box Office Mojo (BOM) Movie Gross**: Contains information on domestic and foreign box office revenues.
   - **Rotten Tomatoes (RT) Movie Info**: Provides metadata about movies, including genres and release dates.
   - **TMDB Movies**: Includes information on movie popularity and ratings.
   - **IMDb Database**: Provides detailed information on directors, actors, writers, and other key personnel involved in the movies.
    

## Data Preparation
In this section, we will load, clean, and merge the datasets to prepare them for analysis.

In [5]:
# Library imports
import pandas as pd
import sqlite3
from zipfile import ZipFile

In [4]:
# Load the datasets
rt_movie_info = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t")
tmdb_movies = pd.read_csv("zippedData/tmdb.movies.csv.gz")
tn_movies = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")